In [58]:
from os import listdir
from torch import load
from lib.env import DATA_PATH
from mne.io import read_raw_edf
import torch
import pandas as pd
from os import makedirs
from lib.ekyn import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

SNEZANA_MICE_IDS_IN_EEG1 = ['21-HET-1','21-HET-2','21-HET-3','21-HET-4','21-HET-5','21-HET-7','21-HET-8','21-KO-1','21-KO-2','21-KO-3','21-KO-4','21-KO-5','21-KO-6','21-KO-7','21-KO-8','21-KO-9','21-KO-10','21-KO-11','21-KO-12','21-WK-1','21-WK-2','21-WK-3','21-WK-4','21-WK-5','21-WK-8','21-WK-9','21-WK-11','21-WK-12','21-WT-1','21-WT-2','21-WT-3','21-WT-4','21-WT-5','21-WT-6','21-WT-7','21-WT-8','21-WT-9','21-WT-10','21-WT-12','381','386']

In [80]:
makedirs(f'{DATA_PATH}/pt_snezana_mice')
raw_snezana_mice_ids = sorted(list(set([id.split('.')[0] for id in listdir(f'{DATA_PATH}/raw_snezana_mice')])))
for id in raw_snezana_mice_ids:
    raw = read_raw_edf(f'{DATA_PATH}/raw_snezana_mice/{id}.edf')
    if id in SNEZANA_MICE_IDS_IN_EEG1:
        eeg = raw.get_data(picks='EEG 1')
    else:
        eeg = raw.get_data(picks='EEG 2')
    eeg = eeg.reshape(-1,5000)
    X = torch.from_numpy(eeg).float()
    df = pd.read_excel(f'{DATA_PATH}/raw_snezana_mice/{id}.xlsx')
    df = df.drop(0).reset_index(drop=True)
    df['Rodent Sleep'] = df['Rodent Sleep'].ffill()
    y = torch.nn.functional.one_hot(torch.from_numpy(pd.Categorical(df['Rodent Sleep']).codes.copy()).long()).float()
    torch.save((X,y),f'{DATA_PATH}/pt_snezana_mice/{id}.pt')

In [60]:
makedirs(f'{DATA_PATH}/pt_snezana_mice_robust')
snezana_mice_ids = sorted(list(set([id.split('.')[0] for id in listdir(f'{DATA_PATH}/pt_snezana_mice')])))

for id in snezana_mice_ids:
    X,y = load_snezana_mice_pt(id)
    Xt = torch.from_numpy(RobustScaler().fit_transform(X)).float()
    torch.save((Xt,y),f'{DATA_PATH}/pt_snezana_mice_robust/{id}.pt')